In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import plotly.graph_objects as go


In [ ]:
import nltk
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel
import pyLDAvis.gensim

In [ ]:
dataset=pd.read_csv('../input/elon-musks-tweets/data_elonmusk.csv',encoding='latin1')
dataset.head(5)

In [ ]:
dataset.drop(['row ID'],axis=1,inplace=True)

In [ ]:
dataset.info()

In [ ]:
#converting the time into datetime
#dataset['Time']=pd.to_datetime(dataset['Time'],format='%y-%m-%d %H:%M:%S')

In [ ]:
fig=go.Figure([go.Scatter(x=dataset['Time'])])
fig.show()

In [ ]:
dataset['Tweet'][5]

In [ ]:
#Text Cleaning

#lowercasing
dataset['Tweet']=dataset['Tweet'].str.lower()
print(dataset['Tweet'][5],'\n')
#removing URl links
dataset['Tweet']=dataset['Tweet'].str.replace('http\S+|www.\S+','',case=False)
print(dataset['Tweet'][5],'\n')
#removing non ascii characters
dataset['Tweet']=dataset['Tweet'].str.encode('ascii','ignore').str.decode('ascii')
print(dataset['Tweet'][5],'\n')
#removing punctations
dataset['Tweet']=dataset['Tweet'].str.replace('[^\w\s]','')
print(dataset['Tweet'][5],'\n')
#removing digits
dataset['Tweet']=dataset['Tweet'].str.replace('\d+','')

#
dataset['Tweet']=dataset['Tweet'].str.replace('rt+','')
print(dataset['Tweet'][5],'\n')
#tokenizing the text
dataset['Tweet']=dataset.apply(lambda x:nltk.word_tokenize(x['Tweet']),axis=1)
print(dataset['Tweet'][5],'\n')
#lemmatizing the words

dataset['Tweet']=dataset['Tweet'].apply(lambda x: (lemmatizer.lemmatize(word)  for word in x))
print(dataset['Tweet'][5],'\n')
#removing stopwords
dataset['Tweet']=dataset['Tweet'].apply(lambda x: ' '.join(word for word in x if word not in (stopwords)))
print(dataset['Tweet'][5],'\n')

dataset['Tweet_dict']=dataset.apply(lambda x:nltk.word_tokenize(x['Tweet']),axis=1)

In [ ]:
dataset['Tweet'][:5]

In [ ]:
#creating dictionary from data
dictionary=corpora.Dictionary(dataset['Tweet_dict'])

In [ ]:
doc_term_matrix=[dictionary.doc2bow(tokens) for tokens in dataset['Tweet_dict']]


In [ ]:
tfidf=TfidfModel(doc_term_matrix)

In [ ]:
corpus_tfidf=tfidf[doc_term_matrix]

In [ ]:
corpus_tfidf

In [ ]:
topics=10

lda=gensim.models.ldamodel.LdaModel(corpus_tfidf,id2word=dictionary,num_topics=topics)

In [ ]:
lda.show_topics()

In [ ]:
pyLDAvis.enable_notebook()
corpora_lda=lda[corpus_tfidf]
topic_display=pyLDAvis.gensim.prepare(lda,corpora_lda,dictionary,mds='tsne')
topic_display